## Imports

In [16]:
from pathlib import Path
import os
import numpy as np

from flood_adapt import FloodAdapt
from flood_adapt.misc.exceptions import AlreadyExistsError
from flood_adapt.objects import (
    EventSet,
    Projection, 
    PhysicalProjection, 
    SocioEconomicChange, 
    Scenario, 
    Strategy,
    )
from flood_adapt.objects.forcing.unit_system import UnitTypesLength, UnitfulLength, UnitfulLengthRefValue, VerticalReference
from flood_adapt.config.config import Settings


## Step 1a: Inputs (Event set and SLR)

Provide the event set and the range of SLR values to be calculated. 

In [10]:
DATA_DIR = Path(r"c:\Users\winter_ga\Offline_data\FloodAdapt-WorkingDatabase\Charleston\4_FloodAdapt\Database")
site = "charleston_beta_release"
name_event_set = "event_set_coastal"

slr = np.arange(0, 1.1, 0.25)  # in meters


## Step 1b: Reading FloodAdapt database and event set

In [11]:
# Setup FloodAdapt
fn_event_set = DATA_DIR / site / "input" / "events" / name_event_set / f"{name_event_set}.toml"
settings = Settings(
    DATABASE_ROOT=DATA_DIR,
    DATABASE_NAME=site
)
fa = FloodAdapt(database_path=settings.database_path)

event_set = EventSet.load_file(fn_event_set)

2025-11-20 05:01:12 PM - FloodAdapt.Database - INFO - Initializing database to charleston_beta_release at c:\users\winter_ga\offline_data\floodadapt-workingdatabase\charleston\4_floodadapt\database


2025-11-20 05:01:20 PM - FloodAdapt.SfincsAdapter - INFO - Deleted simulation folder: c:\Users\winter_ga\Offline_data\FloodAdapt-WorkingDatabase\Charleston\4_FloodAdapt\Database\charleston_beta_release\output\scenarios\ian2_current_no_measures\Flooding\simulations\overland
2025-11-20 05:01:20 PM - FloodAdapt.Database - INFO - Deleted simulation folder: c:\Users\winter_ga\Offline_data\FloodAdapt-WorkingDatabase\Charleston\4_FloodAdapt\Database\charleston_beta_release\output\scenarios\ian2_current_no_measures\Flooding\simulations\offshore
2025-11-20 05:01:20 PM - FloodAdapt.SfincsAdapter - INFO - Deleted simulation folder: c:\Users\winter_ga\Offline_data\FloodAdapt-WorkingDatabase\Charleston\4_FloodAdapt\Database\charleston_beta_release\output\scenarios\ian_current_no_measures\Flooding\simulations\overland
2025-11-20 05:01:20 PM - FloodAdapt.Database - INFO - Deleted simulation folder: c:\Users\winter_ga\Offline_data\FloodAdapt-WorkingDatabase\Charleston\4_FloodAdapt\Database\charleston_

## Step 2a: Create projections and strategies

In [ ]:
# create projections for all SLR values
projections = []
se_change = SocioEconomicChange(population_growth_existing=0,
                          economic_growth=0)
for nn,s in enumerate(slr):
    phys_proj = PhysicalProjection(sea_level_rise=UnitfulLength(value=s, units=UnitTypesLength.meters))
    proj = Projection(
        name=f"SLR_{nn}",
        description=f"Sea level rise of {s:.2f} meters",
        physical_projection=phys_proj, 
        socio_economic_change=se_change
    )
    
    try:
        fa.save_projection(proj)
    except AlreadyExistsError:
        print(f"Projection {proj.name} already exists in database")
    projections.append(proj)

projections

[Projection(name='SLR_0', description='Sea level rise of 0.00 meters', physical_projection=PhysicalProjection(sea_level_rise=UnitfulLength(value=0.0, units=UnitTypesLength.meters), subsidence=UnitfulLength(value=0.0, units=UnitTypesLength.meters), rainfall_multiplier=1.0, storm_frequency_increase=0.0), socio_economic_change=SocioEconomicChange(population_growth_existing=0.0, economic_growth=0.0, population_growth_new=0.0, new_development_elevation=None, new_development_shapefile=None)),
 Projection(name='SLR_1', description='Sea level rise of 0.25 meters', physical_projection=PhysicalProjection(sea_level_rise=UnitfulLength(value=0.25, units=UnitTypesLength.meters), subsidence=UnitfulLength(value=0.0, units=UnitTypesLength.meters), rainfall_multiplier=1.0, storm_frequency_increase=0.0), socio_economic_change=SocioEconomicChange(population_growth_existing=0.0, economic_growth=0.0, population_growth_new=0.0, new_development_elevation=None, new_development_shapefile=None)),
 Projection(nam

## Step 2b: Create strategy

In [17]:
# Create a strategy object
strategy = Strategy(
    name="no_measures",
    description="No measures",
    measures=[],
)
try: 
    fa.save_strategy(strategy)
except AlreadyExistsError:
    print("Strategy already exists in database")

strategy

Strategy already exists in database


Strategy(name='no_measures', description='No measures', measures=[])

## Step 3: Set up scenarios 

In [ ]:
scenarios = []
for proj in projections:
    for event in event_set.sub_events:
        scen = Scenario(
            name=f"{proj.name}_{event.name}_{strategy.name}",
            description=f"Scenario with {proj.description}, Event {event.name}, and {strategy.description}",
            event=event.name,
            projection=projection.name,
            strategy=strategy.name,
        )
        scenarios.append(scen)

## Step 4: Run scenarios

## Step 5: Prepare look up tables